In [1]:
import os
import sys

In [2]:
#sys.path.insert(0, '/Users/marusy/Programming/another-fem-framework/')
sys.path.insert(0, '/home/lshtanko/Programming/another-fem-framework/')
#sys.path.insert(0, '/Users/leonshting/Programming/Schlumberger/fem-framework/')

In [3]:
import numpy as np
from interpolant.cell_to_points import Cell2PointsConverter
from interpolant.strict import StrictSolve
from interpolant.opt import OptSolve
from common.helpers import *
from interpolant.helpers import *
from matplotlib import pyplot as plt
%matplotlib inline

In [4]:
np.set_printoptions(linewidth=500, precision=3)

In [5]:
c2p = Cell2PointsConverter(orders=[[3],[3,3]])

In [18]:
_p=c2p.get_points()
_p

[array([ 0.   ,  0.276,  0.724,  1.   ]),
 array([ 0.   ,  0.138,  0.362,  0.5  ,  0.638,  0.862,  1.   ])]

In [9]:
c2p.get_diag_weights()

[array([[ 0.083,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.417,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.417,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.083]]),
 array([[ 0.042,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.208,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.208,  0.   ,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.083,  0.   ,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.208,  0.   ,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.208,  0.   ],
        [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.042]])]

In [28]:
_w=c2p.get_weights()
o=5
(1/o-np.sum(_w[0]*_p[0]**(o-1)))*o, (1/o-np.sum(_w[1]*_p[1]**(o-1)))*o

(-8.4624568602587047e-08, -1.2903394186469797e-08)

In [31]:
strict_solve = StrictSolve(points=c2p.get_points(), weights=c2p.get_diag_weights(), verbose=True)

In [ ]:
c2p.get_points()

In [30]:
strict_solve.fit_strict(max_power=strict_solve.get_max_pow(unrisky=True), method='sympy')

Number of equations: 11
Number of degrees of freedom: 28
Strictly stitched_power: 0
Start solving strict part
End solving strict part
NullspaceRank: 18


In [ ]:
interps = strict_solve.get_interpolants()
wgths = strict_solve.get_weight_matrices()

In [ ]:
opt_solve = OptSolve(matrices=interps,
                     points=c2p.get_points(),
                     strict_power=strict_solve.get_max_pow(unrisky=True),
                     weights_matrices=wgths,
                     verbose=True
                    )

In [ ]:
opt_solve.fit_opt(constraints=True, tr_fun_option='chebyshev', orders_forwards=2)

In [ ]:
opt_solve.restart_opt(tolerance=1e-3)

In [ ]:
I_rl = np.array(opt_solve.I_rl_subbed).astype(np.float64)
I_lr = np.array(opt_solve.I_lr_subbed).astype(np.float64)

In [ ]:
tr_f = np.vectorize(lambda x: (x-0.5)*x*(x-1))

In [ ]:
eps_l, eps_r = convergence_test(I_lr=new_int1,
                                I_rl=new_int2,
                                p_l=opt_solve.points[0],
                                p_r=opt_solve.points[1],
                                init_size=(0,1),
                                tr_f=tr_f,
                                weights=[np.array(strict_solve.W_l), np.array(strict_solve.W_r)],
                                div_lim=7)

In [ ]:
explosion_test(I_lr=I_lr,
               I_rl=I_rl,
               p_l=opt_solve.points[0],
               p_r=opt_solve.points[1],
               tr_f=tr_f,
               reps=3)

In [ ]:
plt.semilogy(range(len(eps_l)), eps_l)
plt.semilogy(range(len(eps_r)), eps_r)
plt.semilogy(np.linspace(0, len(eps_r)), float(eps_r[0])*np.exp(-np.linspace(0, len(eps_r))))
plt.semilogy(np.linspace(0, len(eps_r)), float(eps_r[0])*np.exp(-2*np.linspace(0, len(eps_r))))

In [ ]:
np.linalg.eigvals(np.dot(new_int1, new_int2))

In [ ]:
np.linalg.eigvals(np.dot(I_lr, I_rl))

In [ ]:
new_int1 = (I_lr + I_lr[::-1,::-1])/2

In [ ]:
new_int2 = (I_rl + I_rl[::-1, ::-1])/2

In [ ]:
c2p.get_weights()[0]

In [ ]:
np.dot(I_rl, c2p.get_weights()[1]).sum()

In [ ]:
np.dot(np.dot(c2p.get_diag_weights()[1], new_int1), np.linalg.inv(c2p.get_diag_weights()[0])).T

In [ ]:
new_int2

In [ ]:
new_int2= np.array([[ 0.47913998,  0.6452334 , -0.21919967,  0.02834773,  0.39265603,
        -0.39775608,  0.07157862],
       [ 0.03097077,  0.37368352,  0.51547025,  0.09433045, -0.05016152,
         0.07682102, -0.04111448],
       [-0.04111448,  0.07682102, -0.05016152,  0.09433045,  0.51547025,
         0.37368352,  0.03097077],
       [ 0.07157862, -0.39775608,  0.39265603,  0.02834773, -0.21919967,
         0.6452334 ,  0.47913998]])

In [ ]:
new_int1 = np.array([[ 0.95827995,  0.30970766, -0.41114484,  0.14315723],
       [ 0.25809336,  0.74736704,  0.15364203, -0.15910243],
       [-0.08767987,  1.03094049, -0.10032303,  0.15706241],
       [ 0.02834773,  0.47165227,  0.47165227,  0.02834773],
       [ 0.15706241, -0.10032303,  1.03094049, -0.08767987],
       [-0.15910243,  0.15364203,  0.74736704,  0.25809336],
       [ 0.14315723, -0.41114484,  0.30970766,  0.95827995]])

In [ ]:
new_int1

In [ ]:
new_int2

In [ ]:
new_int1 = np.array([[ 1.   ,  0.   ,  0.   ],
       [ 0.375,  0.75 , -0.125],
       [ 0.   ,  1.   ,  0.   ],
       [-0.125,  0.75 ,  0.375],
       [ 0.   ,  0.   ,  1.   ]])



In [ ]:
new_int2 = np.dot(np.dot(c2p.get_diag_weights()[1], new_int1), np.linalg.inv(c2p.get_diag_weights()[0])).T

In [ ]:
new_int1

In [ ]:
new_int2

In [ ]:
new_int1 = normalize((new_int1 + new_int2.T)/2)

In [ ]:
def normalize(v):
    norm=np.linalg.norm(v, ord=1, axis=-1)
    return (v.T/norm).T

In [ ]:
from common.polynom_factory import polynom_factory

def get_href_constrain_matrix(order, inversed=False, distribution='uniform'):
    glob_func_on_loc_mesh = np.zeros((order+1, order+1))
    funcs_prim, glob_grid = polynom_factory(dim=1, order=order,size=(0,2), distribution=distribution)
    funcs, local_grid = polynom_factory(dim=1, order=order,size=(0,1), distribution=distribution)
    for num1, i in enumerate(local_grid):
        for num2, prim_f in enumerate(funcs_prim):
            glob_func_on_loc_mesh[num1, num2] = prim_f.subs({'x_1':i[0]})
    if(inversed):
        return np.linalg.inv(glob_func_on_loc_mesh)
    else:
        return glob_func_on_loc_mesh

In [ ]:
a =get_href_constrain_matrix(distribution='globatto', order=4)
b = np.vstack((a, a[1:][::-1,::-1]))

In [ ]:
a[1:][::-1, ::-1]

In [ ]:
a[1:][::-1, ::-1]

In [ ]:
b

In [ ]:
c2p.get_weights()

In [ ]:
from common import polynom_factory